In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [ ]:
from google.colab import files
#path_to_file = list(files.upload().keys())[0]

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
#print(vocab)
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
def int_to_text(ints):
  try:
    #print(ints)
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-4.6815919e-03  3.7439226e-03  4.8695426e-03 ...  2.7518282e-03
   -1.2373817e-03 -5.1891957e-03]
  [-7.1116928e-03  2.4388877e-03 -2.7195443e-03 ... -1.6120905e-03
   -3.2213735e-03 -1.6790213e-03]
  [-3.8629407e-03  2.2983672e-03 -2.0372362e-03 ... -2.4623112e-03
   -9.6476823e-04  9.2965353e-04]
  ...
  [-1.2632029e-02 -1.5756049e-03  8.3591361e-03 ...  7.2647505e-03
    2.5735884e-03 -4.4383113e-03]
  [-8.5563501e-03 -1.0930412e-03  6.8003647e-03 ...  4.7730552e-03
    3.4861271e-03 -1.8534412e-03]
  [-7.7565745e-03 -1.3080130e-02 -1.9308982e-03 ...  7.4420981e-03
    5.6326180e-03 -5.2800491e-03]]

 [[ 5.9712012e-03  4.5018592e-03  1.4695895e-03 ...  2.2031851e-03
   -4.6121422e-03  3.6729949e-03]
  [ 1.2774410e-04  8.4255831e-03  6.4125750e-03 ...  1.9652119e-03
   -1.2341477e-03  5.0920630e-03]
  [-3.2079010e-04  5.6550996e-03  1.0030738e-02 ...  3.7912349e-03
    2.4715397e-03  7.1697019e-04]
  ...
  [-7.2304429e-03 -4.1282098e-03  1.4459397e-02 ... -5.6593679e

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00468159  0.00374392  0.00486954 ...  0.00275183 -0.00123738
  -0.0051892 ]
 [-0.00711169  0.00243889 -0.00271954 ... -0.00161209 -0.00322137
  -0.00167902]
 [-0.00386294  0.00229837 -0.00203724 ... -0.00246231 -0.00096477
   0.00092965]
 ...
 [-0.01263203 -0.0015756   0.00835914 ...  0.00726475  0.00257359
  -0.00443831]
 [-0.00855635 -0.00109304  0.00680036 ...  0.00477306  0.00348613
  -0.00185344]
 [-0.00775657 -0.01308013 -0.0019309  ...  0.0074421   0.00563262
  -0.00528005]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-4.6815919e-03  3.7439226e-03  4.8695426e-03 -3.3727940e-04
  3.2281198e-03 -2.6651658e-04  2.8959750e-03 -5.6426888e-03
  4.8216940e-03  4.2806263e-05 -4.8583318e-03  1.2096581e-03
 -2.7677100e-03 -2.2116841e-03  5.4161469e-03 -2.1009860e-03
 -2.7926131e-03 -1.5248366e-03 -1.0337454e-02  4.6554580e-04
  1.8401269e-03  6.2545766e-03 -8.1639979e-03  6.1668857e-04
 -8.7611042e-03  1.4593744e-03  4.5660068e-03  5.1991944e-03
  2.0836825e-03  2.1776988e-03  4.8540961e-03  2.6007558e-03
 -2.3722029e-03  3.4262827e-03  2.7506372e-03 -3.1821977e-03
  4.4646312e-04 -3.9158124e-03 -2.6573469e-03 -3.5640076e-03
  1.3068549e-03 -1.0277744e-03 -8.3314744e-04 -1.1126553e-03
  1.6631335e-03 -4.4461139e-03  3.4402222e-03 -1.9849213e-03
  1.2478990e-03 -5.0615729e-03 -3.2595524e-03  2.2337409e-03
 -2.9287417e-05 -4.6740300e-03 -2.1397832e-03  2.3123187e-03
 -5.1874318e-05  3.0719051e-03  1.3971515e-04  4.2776340e-03
 -4.0948917e-03  1.9411938e-03  2.7518282e-03 -1.2373817e-03
 -5.189195

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"BiLFDqU&O$gzIUNa:gBQzpO?f\nMA; 3IGjz!evihxjeDKlKvpqRzINgWEQnJXGU;GWiRGJJ-:pFfOLAztXOR\nT','3uoUfSEwU I"

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 23s 76ms/step - loss: 2.5638
Epoch 2/10
172/172 [==============================] - 15s 81ms/step - loss: 1.8664
Epoch 3/10
172/172 [==============================] - 16s 81ms/step - loss: 1.6271
Epoch 4/10
172/172 [==============================] - 16s 76ms/step - loss: 1.4985
Epoch 5/10
172/172 [==============================] - 14s 74ms/step - loss: 1.4200
Epoch 6/10
172/172 [==============================] - 16s 79ms/step - loss: 1.3652
Epoch 7/10
172/172 [==============================] - 16s 81ms/step - loss: 1.3214
Epoch 8/10
172/172 [==============================] - 15s 78ms/step - loss: 1.2817
Epoch 9/10
172/172 [==============================] - 15s 77ms/step - loss: 1.2469
Epoch 10/10
172/172 [==============================] - 15s 76ms/step - loss: 1.2108


In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))